In [ ]:
%reset
import csv
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.ndimage import median_filter
import numpy as np
import pandas as pd
from scipy import signal
import statistics as stats
import matplotlib.dates as mdates
from matplotlib.transforms import Transform
from matplotlib.ticker import (
    AutoLocator, AutoMinorLocator)
from scipy import stats
import scipy.sparse as sparse

In [ ]:
def weighted_average(distribution, weights):
    return round(sum([distribution[i]*weights[i] for i in range(len(distribution))])/sum(weights),2)

def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1]]
                data.append(columns)

    return data

def import_csv2(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8],
                           row[9], row[10] ]
                data.append(columns)

    return data

def import_csv3(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8],
                           row[9], row[10], row[11] ]
                data.append(columns)

    return data

def import_csv4(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1], row[2], row[3], row[4]]
                data.append(columns)

    return data

In [ ]:
tcp_cca = (['TcpCubic', 'TcpBbr', 'TcpQtOptimal'])
simset = (['1.2xminRTT', '1.5xminRTT'])
BDP = 172
buffer_size = 518 
print(buffer_size, tcp_cca, simset)

In [ ]:
num_flows = 1
cwnd = {}
agg_cwnd = {}
inflight = {}
rtt = {}

for algorithm in tcp_cca:
    cwnd[algorithm] = {}
    agg_cwnd[algorithm] = {}
    inflight[algorithm] = {}
    rtt[algorithm] = {}

    for sim in simset:
        cwnd[algorithm][sim] = {}
        inflight[algorithm][sim] = {}
        rtt[algorithm][sim] = {}
        agg_cwnd[algorithm][sim] = {'time': list(), 'agg_cwnd': list(), 'agg_inflight': list(),
                'aveRtt': list(), 'agg_insThruput': list(), 'agg_aveThruput': list(), 
                'agg_insGoodput': list(), 'agg_aveGoodput': list(), 'agg_TxPkts': list(), 
                'agg_ReTxPkts': list(), 'fairness': list(), 'agg_cost': list()}
        
        for i in range(num_flows):
            cwnd[algorithm][sim][i] = {'time': list(), 'cwnd': list(), 'inflight': list(),
                'rtt': list(), 'insThruput': list(), 'aveThruput': list(), 
                'insGoodput': list(), 'aveGoodput': list(), 'TxPkts': list(), 'ReTxPkts': list(), 
                'cost': list()}
                                       
            inflight[algorithm][sim][i] = {'time': list(), 'inflight': list(), 'cwnd': list(), 
                                           'rtt': list(), 'cost': list()}
            rtt[algorithm][sim][i] = {'time': list(), 'rtt': list()}

cwnd_files = []
agg_cwnd_files = []
inflight_files = []
rtt_files = []
                                               
for algorithm in tcp_cca:
    print(algorithm)
    cwnd_sim_files = []
    agg_cwnd_sim_files = []
    inflight_sim_files = []
    rtt_sim_files = []
    for sim in simset:
        agg_cwnd_sim_files.append("../ns-allinone-3.40/ns-3.40/results/qt_optimal_cost_" + str(sim) + "/" + 
                     str(num_flows) + "-flows/10Mbps-100ms/" + str(buffer_size) +
                     "p-btlqueue/" + str(algorithm) + "/" + 
                     algorithm + "-" + str(num_flows) + "-flows-10Mbps-100ms-" + str(buffer_size) +
                     "p-" + "Agg" + "-cwnd.data")
        
        cwnd_flows_files = []
        inflight_flows_files = []
        rtt_flows_files = []
        for i in range(num_flows):
            cwnd_flows_files.append("../ns-allinone-3.40/ns-3.40/results/qt_optimal_cost_" + str(sim) + "/" + 
                     str(num_flows) + "-flows/10Mbps-100ms/" + str(buffer_size) +
                     "p-btlqueue/" + str(algorithm) + "/" + 
                     algorithm + "-" + str(num_flows) + "-flows-10Mbps-100ms-" + str(buffer_size) +
                     "p--flow" + str(i) + "-cwnd.data")
            inflight_flows_files.append("../ns-allinone-3.40/ns-3.40/results/qt_optimal_cost_" + str(sim) + "/" + 
                     str(num_flows) + "-flows/10Mbps-100ms/" + str(buffer_size) +
                     "p-btlqueue/" + str(algorithm) + "/" + 
                     algorithm + "-" + str(num_flows) + "-flows-10Mbps-100ms-" + str(buffer_size) +
                     "p--flow" + str(i) + "-inflight.data")
            rtt_flows_files.append("../ns-allinone-3.40/ns-3.40/results/qt_optimal_cost_" + str(sim) + "/" + 
                     str(num_flows) + "-flows/10Mbps-100ms/" + str(buffer_size) +
                     "p-btlqueue/" + str(algorithm) + "/" + 
                     algorithm + "-" + str(num_flows) + "-flows-10Mbps-100ms-" + str(buffer_size) +
                     "p--flow" + str(i) + "-rtt.data")
        
        cwnd_sim_files.append(cwnd_flows_files)
        inflight_sim_files.append(inflight_flows_files)
        rtt_sim_files.append(rtt_flows_files)
        
    cwnd_files.append(cwnd_sim_files)
    inflight_files.append(inflight_sim_files)
    rtt_files.append(rtt_sim_files)
    
    agg_cwnd_files.append(agg_cwnd_sim_files)


In [ ]:
for algorithm  in tcp_cca:
    index_alg = tcp_cca.index(algorithm)
    print(algorithm)
    
    for sim in simset:
        index_sim = simset.index(sim)
        agg_cwnd_data_in = import_csv3(agg_cwnd_files[index_alg][index_sim])
        for row in agg_cwnd_data_in: 
            agg_cwnd[algorithm][sim]['time'].append({row[0]})
            agg_cwnd[algorithm][sim]['agg_cwnd'].append({row[1]})
            agg_cwnd[algorithm][sim]['agg_inflight'].append({row[2]})
            agg_cwnd[algorithm][sim]['aveRtt'].append({row[3]})
            agg_cwnd[algorithm][sim]['agg_insThruput'].append({row[4]})
            agg_cwnd[algorithm][sim]['agg_aveThruput'].append({row[5]})
            agg_cwnd[algorithm][sim]['agg_insGoodput'].append({row[6]})
            agg_cwnd[algorithm][sim]['agg_aveGoodput'].append({row[7]})
            agg_cwnd[algorithm][sim]['agg_TxPkts'].append({row[8]})
            agg_cwnd[algorithm][sim]['agg_ReTxPkts'].append({row[9]})
            agg_cwnd[algorithm][sim]['fairness'].append({row[10]})
            agg_cwnd[algorithm][sim]['agg_cost'].append({row[11]})

        for i in range(num_flows):
            cwnd_data_in = import_csv2(cwnd_files[index_alg][index_sim][i])
            inflight_data_in = import_csv4(inflight_files[index_alg][index_sim][i])
            rtt_data_in = import_csv(rtt_files[index_alg][index_sim][i])

            for row in cwnd_data_in:
                cwnd[algorithm][sim][i]['time'].append({row[0]})
                cwnd[algorithm][sim][i]['cwnd'].append({row[1]})
                cwnd[algorithm][sim][i]['inflight'].append({row[2]})
                cwnd[algorithm][sim][i]['rtt'].append({row[3]})
                cwnd[algorithm][sim][i]['insThruput'].append({row[4]})
                cwnd[algorithm][sim][i]['aveThruput'].append({row[5]})
                cwnd[algorithm][sim][i]['insGoodput'].append({row[6]})
                cwnd[algorithm][sim][i]['aveGoodput'].append({row[7]})
                cwnd[algorithm][sim][i]['TxPkts'].append({row[8]})
                cwnd[algorithm][sim][i]['ReTxPkts'].append({row[9]})
                cwnd[algorithm][sim][i]['cost'].append({row[10]})
            
            for row in inflight_data_in:
                inflight[algorithm][sim][i]['time'].append({row[0]})
                inflight[algorithm][sim][i]['inflight'].append({row[1]})
                inflight[algorithm][sim][i]['cwnd'].append({row[2]})
                inflight[algorithm][sim][i]['rtt'].append({row[3]})
                inflight[algorithm][sim][i]['cost'].append({row[4]})

            for row in rtt_data_in:
                rtt[algorithm][sim][i]['time'].append({row[0]})
                rtt[algorithm][sim][i]['rtt'].append({row[1]})


In [ ]:
cwnd_time = []
cwnd_value = []
inflight_value = []
rtt_value = []
insThruput_value = []
aveThruput_value = []
insGoodput_value = []
aveGoodput_value = []
cost_value = []

inflight2_time = []
inflight2_value = []
cwnd2_value = []
rtt2_value = []
cost2_value = []
rtt3_time = []
rtt3_value = []

agg_cwnd_time = []
agg_cwnd_value = []
agg_inflight_value = []
agg_rtt_value = []
agg_insThruput_value = []
agg_aveThruput_value = []
agg_insGoodput_value = []
agg_aveGoodput_value = []
fairness_value = []
agg_cost_value = []

for algorithm in tcp_cca:
    print(algorithm)
    cwnd_time_ = []
    cwnd_value_ = []
    inflight_value_ = []
    rtt_value_ = []
    insThruput_value_ = []
    aveThruput_value_ = []
    insGoodput_value_ = []
    aveGoodput_value_ = []
    cost_value_ = []
    
    inflight2_time_ = []
    inflight2_value_ = []
    cwnd2_value_ = []
    rtt2_value_ = []
    cost2_value_ = []
    rtt3_time_ = []
    rtt3_value_ = []
    
    agg_cwnd_time_ = []
    agg_cwnd_value_ = []
    agg_inflight_value_ = []
    agg_rtt_value_ = []
    agg_insThruput_value_ = []
    agg_aveThruput_value_ = []
    agg_insGoodput_value_ = []
    agg_aveGoodput_value_ = []
    fairness_value_ = []
    agg_cost_value_ = []
    
    for sim in simset:
        cwnd_time__ = []
        cwnd_value__ = []
        inflight_value__= []
        rtt_value__ = []
        insThruput_value__ = []
        aveThruput_value__ = []
        insGoodput_value__ = []
        aveGoodput_value__ = []
        cost_value__ = []
        
        inflight2_time__ = []
        inflight2_value__= []
        cwnd2_value__ = []
        rtt2_value__ = []
        cost2_value__ = []
        rtt3_time__ = []
        rtt3_value__ = []

        agg_cwnd_time_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['time']))
        agg_cwnd_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_cwnd']))
        agg_inflight_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_inflight']))
        agg_rtt_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['aveRtt']))
        agg_insThruput_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_insThruput']))  
        agg_aveThruput_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_aveThruput']))  
        agg_insGoodput_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_insGoodput']))  
        agg_aveGoodput_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_aveGoodput'])) 
        fairness_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['fairness'])) 
        agg_cost_value_.append(pd.DataFrame(agg_cwnd[algorithm][sim]['agg_cost'])) 
    
    
        for i in range(num_flows):
            cwnd_time__.append(pd.DataFrame(cwnd[algorithm][sim][i]['time']))
            cwnd_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['cwnd']))
            cost_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['cost']))
            inflight_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['inflight']))
            rtt_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['rtt']))
            insThruput_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['insThruput']))  
            aveThruput_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['aveThruput']))  
            insGoodput_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['insGoodput']))  
            aveGoodput_value__.append(pd.DataFrame(cwnd[algorithm][sim][i]['aveGoodput'])) 

            inflight2_time__.append(pd.DataFrame(inflight[algorithm][sim][i]['time']))
            inflight2_value__.append(pd.DataFrame(inflight[algorithm][sim][i]['inflight']))
            cwnd2_value__.append(pd.DataFrame(inflight[algorithm][sim][i]['cwnd']))
            rtt2_value__.append(pd.DataFrame(inflight[algorithm][sim][i]['rtt']))
            cost2_value__.append(pd.DataFrame(inflight[algorithm][sim][i]['cost']))

        cwnd_time_.append(cwnd_time__)
        cwnd_value_.append(cwnd_value__)
        cost_value_.append(cost_value__)
        inflight_value_.append(inflight_value__)
        rtt_value_.append(rtt_value__)
        insThruput_value_.append(insThruput_value__)
        aveThruput_value_.append(aveThruput_value__)
        insGoodput_value_.append(insGoodput_value__)
        aveGoodput_value_.append(aveGoodput_value__)

        inflight2_time_.append(inflight2_time__)
        inflight2_value_.append(inflight2_value__)
        cwnd2_value_.append(cwnd2_value__)
        rtt2_value_.append(rtt2_value__)
        cost2_value_.append(cost2_value__)
        
    cwnd_time.append(cwnd_time_)
    cwnd_value.append(cwnd_value_)
    cost_value.append(cost_value_)
    inflight_value.append(inflight_value_)
    rtt_value.append(rtt_value_)
    insThruput_value.append(insThruput_value_)
    aveThruput_value.append(aveThruput_value_)
    insGoodput_value.append(insGoodput_value_)
    aveGoodput_value.append(aveGoodput_value_)
    
    inflight2_time.append(inflight2_time_)
    inflight2_value.append(inflight2_value_)
    cwnd2_value.append(cwnd2_value_)
    rtt2_value.append(rtt2_value_)
    cost2_value.append(cost2_value_)
    
    agg_cwnd_time.append(agg_cwnd_time_)
    agg_cwnd_value.append(agg_cwnd_value_)
    agg_inflight_value.append(agg_inflight_value_)
    agg_rtt_value.append(agg_rtt_value_)
    agg_insThruput_value.append(agg_insThruput_value_)  
    agg_aveThruput_value.append(agg_aveThruput_value_)  
    agg_insGoodput_value.append(agg_insGoodput_value_)  
    agg_aveGoodput_value.append(agg_aveGoodput_value_) 
    fairness_value.append(fairness_value_)
    agg_cost_value.append(agg_cost_value_)


In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[0][0][0].to_numpy(dtype=float), cwnd_value[0][0][0].to_numpy(dtype=float)/1448.0, '.-',
        cwnd_time[1][0][0].to_numpy(dtype=float), cwnd_value[1][0][0].to_numpy(dtype=float)/1448,'.-',
        cwnd_time[2][0][0].to_numpy(dtype=float), cwnd_value[2][0][0].to_numpy(dtype=float)/1448,'.-',
        cwnd_time[2][1][0].to_numpy(dtype=float), cwnd_value[2][1][0].to_numpy(dtype=float)/1448,'.-',
        )
#ax1.step(
#        cwnd_time[1].to_numpy(dtype=float), cwnd_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Cwnd (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(150,200)
plt.xlim(0,100)
plt.grid(True)
plt.legend([
            tcp_cca[0], tcp_cca[1], tcp_cca[2] + " - " + simset[0], tcp_cca[2] + " - " + simset[1]
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('cwnd_cross_comparison.png', transparent=True)
